## `Toró Etele 3. Laborfeladat`

1\. Feladat :

> Írjunk **tárolt eljárást**, amely meghatározza azokat a kocsmákat, amelyekben legalább 2 olyan italt árulnak, mint a paraméterként megadott kocsma (@KocsmaNev varchar(30) típusú)! Ha nincs ilyen kocsma, írjunk ki hibaüzenetet, majd szúrjuk be a kocsmát az adatbázisba!
> 
>   
> 
> A feladatot úgy értelmeztem, hogy irassuk ki azokat a kocsmákat, ahol legalább két közös ital található a megadott kocsmával.

In [3]:
CREATE OR ALTER PROCEDURE FirstTask
     @KocsmaNev VARCHAR(30)
AS
BEGIN
    IF NOT EXISTS (SELECT * FROM Kocsmak WHERE Nev = @KocsmaNev) --megnézem létezik-e a megadott kocsma
    BEGIN   --ha nem létezik a megadott kocsma
        PRINT 'A megadott kocsma nem létezik.'
        INSERT INTO Kocsmak (KocsmaID, Nev, Cim, NyitasDatuma) --ha nem, beszúrom
        VALUES(12, @KocsmaNev, 'Valahol itt',  GETDATE())
    END
    ELSE    --ha létezik a megadott kocsma
    BEGIN
        DECLARE @nev VARCHAR(255)
        DECLARE @Kocsmanevek TABLE (Nev VARCHAR(30), Kozos INT) --eredmény tábla

        DECLARE kocsma_cursor CURSOR FOR    --deklarálunk egy kurzort mely sorra veszi a kocsmákat
        SELECT Nev FROM Kocsmak

        OPEN kocsma_cursor
        SET NOCOUNT ON
        FETCH NEXT FROM kocsma_cursor INTO @nev

        WHILE @@FETCH_STATUS = 0
        BEGIN
            IF (@KocsmaNev = @nev)  --amikor ugyanazt a kocsmat dobja a kurzor mint amit megadtunk, lépjen tovább.
            BEGIN
                FETCH NEXT FROM kocsma_cursor INTO @nev
            END

            --Sorra megnézem a táblában hány közös ital van a megadott kocsmával
            DECLARE @kozos INT = (SELECT COUNT(*) FROM (
                SELECT ItalID FROM Arak 
                INNER JOIN Kocsmak ON Kocsmak.KocsmaID = Arak.KocsmaID
                WHERE Nev = @nev
                INTERSECT
                SELECT ItalID FROM Arak 
                INNER JOIN Kocsmak ON Kocsmak.KocsmaID = Arak.KocsmaID
                WHERE Nev = @KocsmaNev
            ) AS Kozos)

            IF (@kozos >=2) --ha megfelelő beszúrom
            BEGIN
                INSERT INTO @Kocsmanevek
                VALUES(@nev, @kozos)
            END

            FETCH NEXT FROM kocsma_cursor INTO @nev
        END

        SELECT * FROM @Kocsmanevek

        CLOSE kocsma_cursor
        DEALLOCATE kocsma_cursor
    END
END
GO

Commands completed successfully.

Total execution time: 00:00:00.027

In [4]:
EXEC dbo.FirstTask @KocsmaNev='Sandra Tweed'


Commands completed successfully.

Total execution time: 00:00:00.032

Nev,Kozos
Frederico Sawley,3
Orren Broadhurst,2
Hedwiga Ledbetter,2


2\. Feladat :

> <span style="background-color: rgba(127, 127, 127, 0.1);">Írjunk&nbsp;</span> **függvényt**<span style="background-color: rgba(127, 127, 127, 0.1);">, melynek bemenő paraméterei: @pBaratNev, @pItaltipusNev. A függvény egy táblában térítse vissza azon kocsmá(k) nevét, címét és a kapható italok nevét és árát mely(ek)et a paraméterként megadott barát kedvel és ahol a megadott italtípusból kapható ital!</span>
> 
> Térítsük vissza a kocsmákat, amit a megadott barát kedvel, és a megadott piák megtalálhatóak.

In [5]:
CREATE OR ALTER FUNCTION secondTask (@pBaratNev VARCHAR(30), @pItalTipusNev VARCHAR(30))
RETURNS TABLE
AS
RETURN
(
    SELECT k.Nev, k.Cim, i.Nev as ItalNeve, a.Ar FROM Kocsmak k
    INNER JOIN Arak a on a.KocsmaID = k.KocsmaID
    INNER JOIN Italok i on i.ItalID = a.ItalID
    INNER JOIN ItalTipusok it ON it.TipusID = i.TipusID
    INNER JOIN Kedvencek f ON f.KocsmaID = k.KocsmaID
    INNER JOIN Baratok b ON b.BaratID = f.BaratID
    WHERE b.Nev = @pBaratNev AND it.TipusNev=@pItalTipusNev
)

GO

Commands completed successfully.

Total execution time: 00:00:00.018

In [6]:
SELECT * FROM dbo.secondTask('Rik Turbard','tea')

(3 rows affected)

Total execution time: 00:00:00.023

Nev,Cim,ItalNeve,Ar
Hedwiga Ledbetter,6 Packers Parkway,Tatra tea,33
Hedwiga Ledbetter,6 Packers Parkway,Fekete tea,29
Sandra Tweed,8 Superior Alley,Fekete tea,22


3\. Feladat :

> <span style="background-color: rgba(127, 127, 127, 0.1);">Írjunk&nbsp;</span> **DELETE triggert**<span style="background-color: rgba(127, 127, 127, 0.1);">, mely akkor aktiválódik, ha törlünk&nbsp;</span> **egy vagy több** <span style="background-color: rgba(127, 127, 127, 0.1);">&nbsp;sort az ItalTipusok táblából! (</span>**Figyelem!** <span style="background-color: rgba(127, 127, 127, 0.1);">&nbsp;A hivatkozási épség megszorítások fenntartása miatt probléma lehet, ha egy olyan italtípust szeretnék törölni, melyhez tartozik ital. Ezt a problémát orvosoljuk a trigger segítségével! Az adott italtípus mindenképp törlődjön a trigger végrehajtásának következtében!)</span>

In [7]:
CREATE OR ALTER TRIGGER ThirdTask
ON ItalTipusok
INSTEAD OF DELETE
AS
BEGIN
    SET NOCOUNT ON
    DELETE Arak FROM Arak 
    INNER JOIN Italok ON Arak.ItalID=Italok.ItalID
    WHERE TipusID IN (SELECT TipusID FROM deleted)

    DELETE Italok WHERE Italok.TipusID IN (SELECT TipusID FROM deleted)

    DELETE ItalTipusok WHERE ItalTipusok.TipusID IN (SELECT TipusID FROM deleted)
END

Commands completed successfully.

Total execution time: 00:00:00.018

4\. Feladat :

> <span style="background-color: rgba(127, 127, 127, 0.1);">Írjunk&nbsp;</span> **INSERT** <span style="background-color: rgba(127, 127, 127, 0.1);"> és </span> **UPDATE** <span style="background-color: rgba(127, 127, 127, 0.1);">&nbsp;triggert az Arak táblára vonatkozóan, amely csak akkor engedélyezi a művelet végrehajtását, ha az árak összértéke 5 és 1500 között marad!</span>

In [2]:
CREATE OR ALTER TRIGGER FourthTask ON Arak
INSTEAD OF UPDATE, INSERT
AS
BEGIN
    SET NOCOUNT ON
    IF(((SELECT SUM(Arak.Ar) FROM Arak)+(SELECT Ar FROM inserted))>5 AND ((SELECT SUM(Arak.Ar) FROM Arak)+(SELECT Ar FROM inserted))<1500)
    BEGIN
        IF( EXISTS (SELECT * FROM deleted))
        BEGIN
            UPDATE Arak
            SET Ar=(SELECT Ar FROM inserted) WHERE Arak.ItalID=(SELECT ItalID FROM inserted) AND Arak.KocsmaID=(SELECT KocsmaID FROM inserted)
        END
        ELSE
        BEGIN
            INSERT INTO Arak (Ar, ItalID, KocsmaID)
            VALUES((SELECT Ar FROM inserted), (SELECT ItalID FROM inserted), (SELECT KocsmaID FROM inserted))
        END
    END
    ELSE
        PRINT 'A megadott modositasok miatt az arak osszege nem esik az [5,1500] intervallumba. Probaljon mas ertekeket'
END

Commands completed successfully.

Total execution time: 00:00:00.015

5\. Feladat :

> <span style="background-color: rgba(127, 127, 127, 0.1);">Írjunk&nbsp;</span> **DELETE triggert**<span style="background-color: rgba(127, 127, 127, 0.1);">, amely nem enged törölni olyan kocsmát, melyet legalább 3-an kedvelnek és a nyitás dátuma 2015 után történt!</span>

In [42]:
CREATE OR ALTER TRIGGER FifthTask ON Kocsmak
INSTEAD OF DELETE
AS
BEGIN
    SET NOCOUNT ON

    IF(EXISTS(SELECT * FROM deleted))
    BEGIN
        DECLARE @ErtekelesekSzama INT, @nyitasDatuma INT

        SELECT @ErtekelesekSzama=COUNT(Kedvencek.BaratID), @nyitasDatuma=YEAR(Kocsmak.NyitasDatuma) FROM Kocsmak 
        INNER JOIN Kedvencek ON Kocsmak.KocsmaID=Kedvencek.KocsmaID
        WHERE Kocsmak.Nev=(SELECT Nev FROM deleted)
        GROUP BY Kocsmak.NyitasDatuma

        IF ( @ErtekelesekSzama > 3 AND @nyitasDatuma >= 2015)
            PRINT 'A kocsma nem torolheto mert 2015 utan nyilt, es tul nagy nepsyerusegnek orvend.'
        ELSE
            BEGIN
                DELETE Arak FROM Arak INNER JOIN Kocsmak ON Arak.KocsmaID=Kocsmak.KocsmaID WHERE Kocsmak.Nev 
                IN (SELECT Nev FROM deleted)

                DELETE Kedvencek FROM Kedvencek INNER JOIN Kocsmak ON Kedvencek.KocsmaID=Kocsmak.KocsmaID WHERE Kocsmak.Nev 
                IN (SELECT Nev FROM deleted)

                DELETE Kocsmak WHERE Kocsmak.Nev 
                IN (SELECT Nev FROM deleted)
            END
    END
END

Commands completed successfully.

Total execution time: 00:00:00.041

6\. Feladat :

> **a.** <span style="background-color: rgba(127, 127, 127, 0.1);"> Hozzunk létre egy új táblát:</span>

>   Arak\_Log(ID, Idopont, Muvelet, KocsmaID, KocsmaNev, ItalID, ItalNev, RegiAr, UjAr),
> 
> ahol Idopont-datetime, Muvelet-varchar(50), RegiAr-int, UjAr-int típusú mezők.

In [33]:
CREATE TABLE Arak_Log (
	ID INT IDENTITY, 
	Idopont DATETIME, 
	Muvelet VARCHAR(50), 
	KocsmaID INT FOREIGN KEY REFERENCES Kocsmak(KocsmaID), 
	KocsmaNev VARCHAR(50), 
	ItalID INT FOREIGN KEY REFERENCES Italok(ItalID),  
	ItalNev VARCHAR(50), 
	RegiAr INT, 
	UjAr INT
)

Commands completed successfully.

Total execution time: 00:00:00.019

> b.<span style="color: rgb(45, 59, 69); font-family: LatoWeb, &quot;Helvetica Neue&quot;, Helvetica, Arial, sans-serif; font-size: 16px; background-color: rgb(255, 255, 255);">Oldjuk meg, hogy a&nbsp;</span> <span style="color: rgb(45, 59, 69); font-family: LatoWeb, &quot;Helvetica Neue&quot;, Helvetica, Arial, sans-serif; font-size: 16px; background-color: rgb(255, 255, 255);">Kocsmak táblából való törlés esetén az Arak_Log tábla&nbsp;</span> <span style="color: rgb(45, 59, 69); font-family: LatoWeb, &quot;Helvetica Neue&quot;, Helvetica, Arial, sans-serif; font-size: 16px; background-color: rgb(255, 255, 255);">KocsmaID</span> <span style="color: rgb(45, 59, 69); font-family: LatoWeb, &quot;Helvetica Neue&quot;, Helvetica, Arial, sans-serif; font-size: 16px; background-color: rgb(255, 255, 255);">mezőjének értéke&nbsp;</span> <span style="color: rgb(45, 59, 69); font-family: LatoWeb, &quot;Helvetica Neue&quot;, Helvetica, Arial, sans-serif; font-size: 16px; background-color: rgb(255, 255, 255);">NULL-ra módosuljon!</span>

In [43]:
CREATE OR ALTER TRIGGER SixthTask ON Kocsmak
INSTEAD OF DELETE
AS
BEGIN
UPDATE Arak_Log SET KocsmaID=NULL WHERE KocsmaID IN (SELECT KocsmaID FROM deleted)
END

: Msg 2111, Level 16, State 1, Procedure SixthTask, Line 1
Cannot create trigger 'SixthTask' on table 'Kocsmak' because an INSTEAD OF DELETE trigger already exists on this object.

Total execution time: 00:00:00.001

**c.** Írjunk **UPDATE triggert**, mely az Arak tábla Ar mezőjének módosításakor beszúr egy új sort a Arak\_Log táblába. Az időpont legyen a módosítás időpontja, művelet szövege: ‘ar növelése’ vagy ‘ar csökkentése’, RegiAr-régi érték, UjAr-új érték.

In [10]:
CREATE OR ALTER TRIGGER SixthTaskPriceInsert ON Arak
AFTER UPDATE
AS 
BEGIN
    DECLARE @regiar INT, @ujar INT, @kID INT, @iID INT
    DECLARE @muv VARCHAR(50), @kname VARCHAR(50), @iname VARCHAR(50)

    SELECT @regiar=Ar FROM deleted
    SELECT @ujar=Ar FROM inserted

    IF (@regiar<@ujar)
    BEGIN
        SET @muv='ar novelese'
    END
    ELSE
    BEGIN
        SET @muv='ar csokkentese'
    END
 
    SELECT @kID=KocsmaID FROM inserted
    SELECT @kname=Kocsmak.Nev FROM inserted
    INNER JOIN Kocsmak ON Kocsmak.KocsmaID=inserted.KocsmaID

    SELECT @iID=ItalID FROM inserted
    SELECT @iname=Italok.Nev FROM inserted
    INNER JOIN Italok ON Italok.ItalID=inserted.KocsmaID

    INSERT INTO Arak_Log(Idopont, Muvelet, RegiAr, UjAr, KocsmaID, ItalID, KocsmaNev, ItalNev)
    VALUES(GETDATE(), @muv, @regiar, @ujar, @kID, @iID, @kname, @iname)
END

Commands completed successfully.

Total execution time: 00:00:00.016